In [ ]:
import os
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

load_dotenv()

os.environ['HUGGINGFACE_API_KEY'] = '...'

In [ ]:
json_file_path = '/content/keskop_answer.json'

with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

questions = []
answers = []

for key, qa_list in data.items():
    for qa in qa_list:
        questions.append(qa['question'])
        answers.append(qa['answer'])

df = pd.DataFrame({'question': questions, 'answer': answers})

In [ ]:
# Improved preprocess function
def preprocess_text(text):
    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('turkish'))
    tokens = word_tokenize(text.lower())
    return ' '.join([token for token in tokens if token not in stop_words])

# Preprocess questions
df['preprocessed_question'] = df['question'].apply(preprocess_text)

documents = [Document(page_content=question, metadata={'answer': answer, 'preprocessed': prep})
             for question, answer, prep in zip(df['question'], df['answer'], df['preprocessed_question'])]

"""
* Text Split – Chunking ( Metin Bölme - Parçalama ) veya veriye göre en uygun işlem uygunalabilir.
- Parçalama, büyük veri bağlamındaki karmaşıklığı ve çeşitliliği azaltmak için büyük veri topluluğunu daha küçük bölümlere ayırma stratejisidir.
  Parçalama, token boyutuna göre daha küçük veri parçalarını böler.
"""

In [ ]:
# Use a more powerful multilingual model
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
"""
* metin gömme modeli daha güçlü ve türkçe kelimeler üstünde daha iyi çalışabilen bir model olabilir.
"""

In [ ]:
vector_store = FAISS.from_documents(documents, embeddings)
"""
* veriye ve modele en uygun veritabanı seçimi yapılabilir.
"""

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token=os.environ['HUGGINGFACE_API_KEY'],
    temperature=0.1,
    model_kwargs={"max_length": 256}
)
"""
* Model seçimi kullanılabilen en parametreli model olması iyi.
* temparature -> 0.1 - 0.3 arası iyi olabilir.
* top_p, veya top_k parametreleri eklenmesi, yanıtların çeşitliliği ve kalitesi üzerinde etkili olabilir.
* max_length değerini artırmak, modelin daha fazla bellek (RAM) ve hesaplama kaynağı kullanmasına neden olur.
  - Büyük uzunluklar, özellikle sınırlı donanım kaynaklarında, bellek hatalarına veya çok yavaş yanıt sürelerine yol açabilir.
  - Daha uzun girişler, yanıt kalitesini artırabilir, ancak aynı zamanda modelin dikkat mekanizmasının zorlanmasına ve dikkat dağılmasına da neden olabilir.
"""


In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
"""
* "k" parametresi, arama sırasında döndürülecek sonuç sayısını belirtir.
  - retriever'ın daha fazla sonuç döndürmesini sağlar, böylece modelin daha fazla bilgiye erişerek daha iyi bir yanıt oluşturması mümkün olur.
  - Ancak, çok fazla sonuç döndürmek de performansı etkileyebilir ve gereksiz bilgilerin değerlendirilmesine yol açabilir.
"""

In [ ]:
custom_prompt = PromptTemplate.from_template(
    "Aşağıdaki talimatları takip ederek soruyu yanıtla:\n\n"
    "{instructions}\n\n"
    "Soru: {question}\n"
    "Bağlam: {context}\n\n"
    "Yanıt:"
)
"""
* prompt iyileştirmesi yapılabilir.
 - Kullanıcı talimatlarını daha açıklayıcı bir şekilde güncelleyerek modelin doğru şekilde yanıt vermesi sağlanabilir.
"""

In [ ]:
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "instructions": lambda _: "Verilen bağlamı kullanarak soruyu yanıtlayın. Eğer soru ve bağlam arasında bir ilişki yoksa, 'Üzgünüm, bu soruyu yanıtlayamıyorum.' deyin."
    }
    | custom_prompt
    | llm
    | StrOutputParser()
)
"""
* instructions belirlenebilir.
"""

In [ ]:
def calculate_similarity(query, document):
    query_embedding = embeddings.embed_query(query)
    doc_embedding = embeddings.embed_query(document.page_content)
    return cosine_similarity([query_embedding], [doc_embedding])[0][0]

def find_most_similar_question(query, documents, threshold=0.7):
    preprocessed_query = preprocess_text(query)
    similarities = [calculate_similarity(preprocessed_query, doc) for doc in documents]
    max_similarity = max(similarities)
    if max_similarity >= threshold:
        most_similar_index = similarities.index(max_similarity)
        return documents[most_similar_index], max_similarity
    return None, max_similarity

"""
* nlp ile soruları ve çıktıları iyileştirebilirsek iyi olur.
"""

In [ ]:
def ask_question(query):
    try:
        most_similar_doc, similarity = find_most_similar_question(query, documents)

        if most_similar_doc:
            result = rag_chain.invoke(query)
            return f"Soru: {query}\nEn benzer soru: {most_similar_doc.page_content}\nBenzerlik skoru: {similarity:.2f}\nCevap: {result}"
        else:
            return f"Üzgünüm. Bu soruya benzer bir soru bulamadım (Benzerlik skoru: {similarity:.2f}). Lütfen başka bir şekilde sormayı deneyin veya farklı bir soru sorun."
    except Exception as e:
        return f"Bir hata oluştu: {str(e)}"

if __name__ == "__main__":
    print("Soru-cevap sistemine hoş geldiniz! Çıkmak için 'q' yazın.")
    while True:
        query = input("\nLütfen bir soru sorun: ")
        if query.lower() == 'q':
            print("Sistemden çıkılıyor. İyi günler!")
            break
        answer = ask_question(query)
        print(answer)